# REFINITIV EIKON - TRADE AND QUOTES 

In [ ]:
import time
import eikon as ek
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# SET VARIABLES

In [ ]:
# GERMAN MARKET SPECIFIC
index = "0#.GDAXI"
home_id = "d."

In [ ]:
# WHICH WEEK OF THE YEAR
week_number = XX

In [ ]:
# DO NOT CHANGE
weeks = 0
days = 5

In [ ]:
# WHICH MTFs
biggest_mtfs = [".DE", ".DXE", ".TQE", ".AQE"]

# SET APP KEY

In [ ]:
app_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
ek.set_app_key(app_key)

In [ ]:
ek.set_timeout(1e6)

# SET TRADING DAYS

In [ ]:
d = f"2024-W{week_number}"
end_date = datetime.strptime(d + '-1', "%Y-W%W-%w")
end_date += + timedelta(days=4)
end_date = end_date.strftime('%Y-%m-%d')

In [ ]:
start_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
start_date_dt = start_date_dt - relativedelta(weeks = weeks, days = days)
start_date = datetime.strftime(start_date_dt, format = "%Y-%m-%d")

In [ ]:
def weekdays_between_dates(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    current_date = start_date
    weekdays_list = []

    while current_date <= end_date:
        if current_date.weekday() < 5:  
            weekdays_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days = 1)

    return weekdays_list

In [ ]:
trading_days = weekdays_between_dates(start_date = start_date, end_date = end_date)
trading_days

# MAKE AND SET PATH TO SAVE

In [ ]:
folder_name = Path(f"TAS_{week_number}")
folder_name.mkdir(parents=True, exist_ok=True) 

In [ ]:
path_to_save = f"./TAS_{week_number}/"

# GET CONSTITUENTS

In [ ]:
index_constituents = pd.read_pickle("index_constituents")
index_constituents

# GET FRAGMENTATION

In [ ]:
fragmentation = pd.read_pickle("fragmentation")
fragmentation

In [ ]:
constituents = fragmentation["ric"].unique()
i = 0
for con in constituents:
    print(f" [{i}] {con}")
    i += 1

# GET TAS DATA

In [ ]:
def get_tas_data(ric, date):
    def fetch_interval(ric, Sdate, Edate, interval):
        # print(f"               Start interval: {Sdate.split('T')[1][:8]} to {Edate.split('T')[1][:8]}")
        tries = 0
        while tries < 3:
            try:                
                raw = ek.get_timeseries(rics = ric, start_date = Sdate, end_date = Edate, interval = "tas", raw_output = True)
                fields = pd.DataFrame(raw["timeseriesData"])["fields"][0]
                column_names = pd.DataFrame(fields)["name"]
                raw_data = pd.DataFrame(raw["timeseriesData"])["dataPoints"][0]
                
                trade_data_inc = pd.DataFrame(data = raw_data, columns = column_names)
                trade_data_inc = trade_data_inc[['TIMESTAMP', 'EXCHTIME', 'TRDPRC_1', 'COUNT', "MMT_CLASS", "TR_TRD_FLG"]]
                trade_data_inc.rename(columns={"TRDPRC_1": "PRICE", "COUNT": "VOLUME"}, inplace = True)
                trade_data_inc = trade_data_inc[~trade_data_inc["PRICE"].isna()]
                
                if len(trade_data_inc) >= 50000:
                    new_Sdate = Sdate
                    new_Edate = Edate
                    new_interval = interval//2
                    print(f"                    Splitting interval for {ric}: {new_Sdate.split('T')[1][:8]} to {new_Edate.split('T')[1][:8]}")
                    mid_date = (datetime.strptime(new_Sdate, "%Y-%m-%dT%H:%M:%S.%fZ") + timedelta(seconds = new_interval)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")                
                    trade_data_1 = fetch_interval(ric, new_Sdate, mid_date, new_interval)
                    trade_data_2 = fetch_interval(ric, mid_date, new_Edate, new_interval)
                    return pd.concat([trade_data_1, trade_data_2], ignore_index=True)
                
                # print(f"               Done interval: {Sdate} to {Edate}")
                return trade_data_inc
            
            except Exception as e:
                tries += 1
                print(f"                    Attempt {tries} failed with error: {e}")
                if tries >= 3:
                    print(f"                    Failed after {tries} attempts.")
                continue
        
        return pd.DataFrame()

    date_conversion = datetime.strptime(date, "%Y-%m-%d")
    Edate_dt = datetime(date_conversion.year, date_conversion.month, date_conversion.day, 15, 59, 59, 999999)
    Sdate_dt = datetime(Edate_dt.year, Edate_dt.month, Edate_dt.day, 7, 0, 0, 0)
    time_difference = (Edate_dt - Sdate_dt).total_seconds()

    trade_data = pd.DataFrame()
    intervals = 3600*9
    Sdate_inc = Sdate_dt
    
    for sec in range(intervals, round(time_difference) + 1, intervals):
        Edate_inc = Sdate_inc + timedelta(seconds = intervals)
        Sdate = Sdate_inc.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        Edate = Edate_inc.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
        trade_data_inc = fetch_interval(ric, Sdate, Edate, intervals)
        trade_data = pd.concat([trade_data, trade_data_inc], ignore_index = True)
        
        Sdate_inc = Edate_inc
    
    return trade_data

In [ ]:
tic = time.time()
day_counter = 0
tot_con_counter = 0

print(f"######################################## Start: GET TAS DATA ########################################\n")

for day in trading_days:
    trade_day = pd.DataFrame()
    day_counter += 1
    con_counter = 0
    print(f"#####################################################################################################")
    print(f"Start day: {day} nr.{day_counter}/{len(trading_days)}")
    print(f"#####################################################################################################\n")
    
    for con in constituents:
        trade_con = pd.DataFrame()
        tot_con_counter += 1
        con_counter += 1
        ric_counter = 0
        print(f"     Start constituent: {con} nr.{con_counter}/{len(constituents)}")

        fragmentation_con = list(fragmentation.loc[fragmentation["ric"] == con, "fragmentation"])

        for ric in fragmentation_con:
            ric_counter += 1
            # print(f"          Start venue: {ric} nr.{ric_counter}/{len(fragmentation_con)}")
            
            trade_ric = get_tas_data(ric, day)
            trade_ric["RIC"] = ric
            trade_con = pd.concat([trade_con, trade_ric], ignore_index = True)

            toc = time.time()
            print(f"          Done venue: {ric} nr.{ric_counter}/{len(fragmentation_con)} in {round(toc-tic,2)} seconds")

        # Save per day per constituent (200 files for 1 week)
        # trade_con.to_csv(path_to_save + "TAS_" + con + "_" + day + ".csv")
        
        toc = time.time()
        print(f"     Done constituent: {con} nr.{con_counter}/{len(constituents)} in {round(toc-tic,2)} seconds\n")
        
        estimated_total_time = ((toc-tic)/tot_con_counter) * (len(trading_days) * len(constituents))
        print("---------------------------------------------------------------------------------------")
        print(f"Progression: {round((tot_con_counter / (len(constituents) * len(trading_days))) * 100, 2)}%")
        print(f"Estimated total time: {estimated_total_time // 3600} hour(s) and {(estimated_total_time % 3600) // 60} minute(s)")
        print(f"Time started: {datetime.fromtimestamp(tic + 2 * 3600)}, Estimated finish: {datetime.fromtimestamp(tic + estimated_total_time + 2 * 3600)}")
        print(f"Estimated time remaining: {(estimated_total_time - (toc - tic)) // 3600} hour(s) and {((estimated_total_time - (toc - tic)) % 3600) // 60} minute(s)")
        print("---------------------------------------------------------------------------------------\n")
        
        trade_day = pd.concat([trade_day, trade_con], ignore_index = True)

    # Save per day (5 files for 1 week)
    trade_day.to_csv(path_to_save + "TAS_" + day + ".csv")
    
    toc = time.time()
    print(f"#####################################################################################################")
    print(f"Done day: {day} nr.{day_counter}/{len(trading_days)} in {round(toc-tic,2)} seconds")
    print(f"#####################################################################################################\n")
    
print(f"######################################## Done: GET TAS DATA #########################################\n")